<a href="https://colab.research.google.com/github/Guenole-tech/Lorentz-IA/blob/main/LorenteQboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from scipy.linalg import expm

# ==========================================
# MODULE 1 : GÉOMÉTRIE RIEMANNIENNE
# ==========================================
class HyperbolicManifold:
    """Gère la métrique et la topologie du code."""
    def __init__(self, size=15):
        # Utilisation d'un graphe d'expanseur pour simuler une courbure négative
        # Ces graphes imitent les propriétés des variétés hyperboliques.
        self.G = nx.margulis_expanders_graph(size)
        self.pos = nx.spring_layout(self.G)

    def metric_tensor(self, node_i, node_j):
        """
        Calcule le poids de l'arête basé sur une métrique pseudo-riemannienne simplifiée.
        ds^2 = g_uv dx^u dx^v
        """
        p1, p2 = np.array(self.pos[node_i]), np.array(self.pos[node_j])
        dist = np.linalg.norm(p1 - p2)
        # Métrique de Poincaré simplifiée : plus on s'éloigne du centre,
        # plus la 'distance' métrique est faible (densité de qubits plus élevée)
        g_uv = 1 / (1 - min(0.99, dist**2))
        return g_uv * dist

# ==========================================
# MODULE 2 : ALGÈBRE DE LIE & STABILISATEURS
# ==========================================
class LieStabilizerCode:
    """Définit les opérateurs de correction via l'algèbre de Lie su(2)."""
    def __init__(self, manifold):
        self.manifold = manifold
        self.qubits = list(manifold.G.nodes())
        self.X, self.Z = self._generate_lie_operators()
        self.stabilizers = self._construct_stabilizers()

    def _generate_lie_operators(self):
        """Matrices de Pauli vues comme générateurs de l'algèbre de Lie."""
        X = np.array([[0, 1], [1, 0]])
        Z = np.array([[1, 0], [0, -1]])
        return X, Z

    def _construct_stabilizers(self):
        """Définit des stabilisateurs sur les cycles de la variété."""
        # On utilise les cycles fondamentaux du graphe pour minimiser l'overhead
        cycles = nx.cycle_basis(self.manifold.G)
        return cycles[:len(self.qubits) // 2] # Ratio d'encodage optimisé

# ==========================================
# MODULE 3 : OPTIMISATION & CONTRÔLE (DÉCODEUR)
# ==========================================
class GeodesicOptimizer:
    """Décodeur utilisant les chemins géodésiques sur la variété."""
    def __init__(self, manifold):
        self.manifold = manifold

    def decode(self, syndrome_nodes):
        """
        Trouve le chemin de correction le plus probable.
        Utilise la métrique g_uv pour pondérer les arêtes.
        """
        weighted_G = self.manifold.G.copy()
        for u, v in weighted_G.edges():
            weighted_G[u][v]['weight'] = self.manifold.metric_tensor(u, v)

        corrections = []
        # Appariement des syndromes par géodésiques minimales
        nodes = list(syndrome_nodes)
        while len(nodes) >= 2:
            start = nodes.pop(0)
            # Trouve le syndrome le plus proche selon la métrique Riemannienne
            end = min(nodes, key=lambda n: nx.shortest_path_length(weighted_G, start, n, weight='weight'))
            nodes.remove(end)
            path = nx.shortest_path(weighted_G, start, end, weight='weight')
            corrections.append(path)
        return corrections

# ==========================================
# MODULE 4 : SIMULATION PRINCIPALE
# ==========================================
def run_quantum_simulation():
    print("1. Initialisation de la variété hyperbolique...")
    manifold = HyperbolicManifold(size=10)

    print("2. Construction du code (Algèbre de Lie)...")
    code = LieStabilizerCode(manifold)
    print(f"   -> Qubits physiques : {len(code.qubits)}")
    print(f"   -> Stabilisateurs actifs : {len(code.stabilizers)}")

    print("3. Injection d'erreurs aléatoires...")
    # Simulation d'erreurs sur deux qubits distants
    error_nodes = [code.qubits[0], code.qubits[7]]

    print("4. Optimisation du contrôle (Décodage Géodésique)...")
    optimizer = GeodesicOptimizer(manifold)
    correction_paths = optimizer.decode(error_nodes)

    # Visualisation
    plt.figure(figsize=(10, 7))
    nx.draw(manifold.G, manifold.pos, node_color='lightblue', with_labels=True, node_size=500)
    # Dessiner les erreurs
    nx.draw_networkx_nodes(manifold.G, manifold.pos, nodelist=error_nodes, node_color='red')
    # Dessiner la correction géodésique
    for path in correction_paths:
        path_edges = list(zip(path, path[1:]))
        nx.draw_networkx_edges(manifold.G, manifold.pos, edgelist=path_edges, edge_color='orange', width=3)

    plt.title("Correction d'Erreur sur Variété Riemannienne (Courbure Négative)")
    plt.show()
    print(f"Correction appliquée sur le chemin géodésique : {correction_paths}")

if __name__ == "__main__":
    run_quantum_simulation()

1. Initialisation de la variété hyperbolique...


AttributeError: module 'networkx' has no attribute 'margulis_expanders_graph'

In [4]:
!pip install qiskit
!pip install qiskit-aer
import matplotlib.pyplot as plt
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer
from qiskit.visualization import plot_histogram
from math import gcd
from pandas import DataFrame
from fractions import Fraction

def qft_dagger(n):
    """Crée un circuit de QFT inverse sur n qubits"""
    qc = QuantumCircuit(n)
    for qubit in range(n//2):
        qc.swap(qubit, n-qubit-1)
    for j in range(n):
        for m in range(j):
            qc.cp(-np.pi/float(2**(j-m)), m, j)
        qc.h(j)
    qc.name = "QFT†"
    return qc

def modular_exponentiation(qc, n_count, a, N):
    """
    Implémente l'exponentiation modulaire contrôlée.
    Note : Pour N=21, a=2, nous utilisons une version simplifiée
    pour éviter l'explosion du nombre de portes.
    """
    for q in range(n_count):
        # On applique a^(2^q) mod N
        # Ici, nous simulons l'effet de la porte contrôlée
        exponent = 2**q
        # Dans un vrai circuit, on insérerait des portes de multiplication modulaire
        qc.append(None, [q] + [i+n_count for i in range(5)]) # Placeholder conceptuel

# --- PARAMÈTRES ---
N = 21
a = 2
n_count = 8  # Nombre de qubits de mesure (précision)

# --- CONSTRUCTION DU CIRCUIT ---
qc = QuantumCircuit(n_count + 5, n_count)

# Initialisation du registre de mesure en superposition
for q in range(n_count):
    qc.h(q)

# Initialisation du registre cible à |1>
qc.x(n_count)

# Exponentiation modulaire (Portes U contrôlées)
# Pour 21, on utilise souvent une approche manuelle pour l'unité U
# Voici la structure de base :
for q in range(n_count):
    # On applique 2^(2^q) mod 21
    # Note : Cette partie est spécifique à la valeur de 'a' et 'N'
    pass

# QFT Inverse
qc.append(qft_dagger(n_count), range(n_count))

# Mesure
qc.measure(range(n_count), range(n_count))

qc.draw(output='text')

┌───┐┌───────┐┌─┐                     
 q_0: ┤ H ├┤0      ├┤M├─────────────────────
      ├───┤│       │└╥┘┌─┐                  
 q_1: ┤ H ├┤1      ├─╫─┤M├──────────────────
      ├───┤│       │ ║ └╥┘┌─┐               
 q_2: ┤ H ├┤2      ├─╫──╫─┤M├───────────────
      ├───┤│       │ ║  ║ └╥┘┌─┐            
 q_3: ┤ H ├┤3      ├─╫──╫──╫─┤M├────────────
      ├───┤│  QFT† │ ║  ║  ║ └╥┘┌─┐         
 q_4: ┤ H ├┤4      ├─╫──╫──╫──╫─┤M├─────────
      ├───┤│       │ ║  ║  ║  ║ └╥┘┌─┐      
 q_5: ┤ H ├┤5      ├─╫──╫──╫──╫──╫─┤M├──────
      ├───┤│       │ ║  ║  ║  ║  ║ └╥┘┌─┐   
 q_6: ┤ H ├┤6      ├─╫──╫──╫──╫──╫──╫─┤M├───
      ├───┤│       │ ║  ║  ║  ║  ║  ║ └╥┘┌─┐
 q_7: ┤ H ├┤7      ├─╫──╫──╫──╫──╫──╫──╫─┤M├
      ├───┤└───────┘ ║  ║  ║  ║  ║  ║  ║ └╥┘
 q_8: ┤ X ├──────────╫──╫──╫──╫──╫──╫──╫──╫─
      └───┘          ║  ║  ║  ║  ║  ║  ║  ║ 
 q_9: ───────────────╫──╫──╫──╫──╫──╫──╫──╫─
                     ║  ║  ║  ║  ║  ║  ║  ║ 
q_10: ───────────────╫──╫──╫──╫──╫──╫──╫──╫─
                     ║  ║  ║  ║  ║  ║  ║  ║ 
q_11: ───────────────╫──╫──╫──╫──╫──╫──╫──╫─
                     ║  ║  ║  ║  ║  ║  ║  ║ 
q_12: ───────────────╫──╫──╫──╫──╫──╫──╫──╫─
                     ║  ║  ║  ║  ║  ║  ║  ║ 
 c: 8/═══════════════╩══╩══╩══╩══╩══╩══╩══╩═
                     0  1  2  3  4  5  6  7

In [7]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from scipy.linalg import expm
import os
from google.colab import files  # Import nécessaire pour télécharger sur ton PC

# --- INITIALISATION PHYSIQUE ---
I = np.eye(2, dtype=complex)
X = np.array([[0, 1], [1, 0]], dtype=complex)
Z = np.array([[1, 0], [0, -1]], dtype=complex)

def calcul_purete(rho):
    return np.real(np.trace(rho @ rho))

# Configuration Figure (Design Start-up)
plt.style.use('dark_background')
fig, ax = plt.subplots(figsize=(12, 7))
ax.set_xlim(0, 200)
ax.set_ylim(0.45, 1.05)
line_std, = ax.plot([], [], color='#ff4b4b', linestyle='--', label='ORDINATEUR STANDARD (Décohérence)', alpha=0.8)
line_lq, = ax.plot([], [], color='#00d1ff', linewidth=3, label='ORDINATEUR LORENTZQ (Boost Relativiste)')
ax.axhline(y=1.0, color='#00ff41', linestyle=':', label='Information Pure (s²=0)')

text_gain = ax.text(105, 0.55, '', fontsize=12, color='white', bbox={'facecolor': '#333333', 'alpha': 0.8})
ax.set_title("LorentzQ : Restauration de la Pureté par Boost de Lorentz", fontsize=16, pad=20)
ax.set_xlabel("Cycles d'Horloge (Temps)", fontsize=12)
ax.set_ylabel("Fidélité / Pureté du Qubit", fontsize=12)
ax.legend(loc='lower left', fontsize=10)
ax.grid(color='gray', linestyle='--', alpha=0.2)

# --- PARAMÈTRES ---
steps = 200
taux_erreur = 0.025

def animate(frame):
    gain = frame * 0.003
    rho_std = np.array([[1, 0], [0, 0]], dtype=complex)
    rho_lq = rho_std.copy()
    f_std, f_lq = [], []

    for i in range(steps):
        bruit = (1 - taux_erreur) * rho_std + taux_erreur * (I / 2.0)
        rho_std = bruit / np.trace(bruit)
        rho_lq = (1 - taux_erreur) * rho_lq + taux_erreur * (I / 2.0)

        x3 = np.real(np.trace(rho_lq @ Z))
        x0 = np.real(np.trace(rho_lq @ I))
        s2 = x0**2 - x3**2

        if s2 > 0.0001:
            eta = np.arctanh(min(0.999, 1 - s2))
            boost = expm(eta * gain * Z)
            rho_lq = boost @ rho_lq @ boost.conj().T
            rho_lq /= np.trace(rho_lq)

        f_std.append(calcul_purete(rho_std))
        f_lq.append(calcul_purete(rho_lq))

    line_std.set_data(range(steps), f_std)
    line_lq.set_data(range(steps), f_lq)
    text_gain.set_text(f"PUISSANCE DU BOOST : {gain:.3f}\nEFFICACITÉ : {((f_lq[-1]-0.5)/0.5)*100:.1f}%")
    return line_std, line_lq, text_gain

# Création
ani = FuncAnimation(fig, animate, frames=100, interval=50, blit=True)

# --- SAUVEGARDE ET TÉLÉCHARGEMENT ---
filename = 'Demo_LorentzQ_Sucrematie.mp4'
print("Génération de la vidéo en cours (veuillez patienter)...")

try:
    # On enregistre d'abord sur le serveur temporaire de Colab
    ani.save(filename, writer='ffmpeg', fps=20)
    print("Vidéo prête. Lancement du téléchargement sur votre ordinateur...")
    # C'est cette ligne qui envoie le fichier sur ton Bureau/Téléchargements
    files.download(filename)
except Exception as e:
    print(f"FFMPEG non détecté, tentative en format GIF...")
    filename_gif = filename.replace('.mp4', '.gif')
    ani.save(filename_gif, writer='pillow', fps=20)
    files.download(filename_gif)

plt.close()

Génération de la vidéo en cours (veuillez patienter)...
Vidéo prête. Lancement du téléchargement sur votre ordinateur...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [8]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from scipy.linalg import expm
import os
from google.colab import files

# --- INITIALISATION PHYSIQUE ---
I = np.eye(2, dtype=complex)
X = np.array([[0, 1], [1, 0]], dtype=complex)
Z = np.array([[1, 0], [0, -1]], dtype=complex)

def calcul_purete(rho):
    return np.real(np.trace(rho @ rho))

# Configuration Figure
plt.style.use('dark_background')
fig, ax = plt.subplots(figsize=(12, 7))
ax.set_xlim(0, 200)
ax.set_ylim(0.45, 1.05)
line_std, = ax.plot([], [], color='#ff4b4b', linestyle='--', label='ORDINATEUR STANDARD (Décohérence)', alpha=0.8)
line_lq, = ax.plot([], [], color='#00d1ff', linewidth=3, label='ORDINATEUR LORENTZQ (Boost Relativiste)')
ax.axhline(y=1.0, color='#00ff41', linestyle=':', label='Information Pure (s²=0)')

text_gain = ax.text(105, 0.55, '', fontsize=12, color='white', bbox={'facecolor': '#333333', 'alpha': 0.8})
ax.set_title("LorentzQ : Restauration de la Pureté par Boost de Lorentz", fontsize=16, pad=20)
ax.set_xlabel("Cycles d'Horloge (Temps)", fontsize=12)
ax.set_ylabel("Fidélité / Pureté du Qubit", fontsize=12)
ax.legend(loc='lower left', fontsize=10)
ax.grid(color='gray', linestyle='--', alpha=0.2)

# --- PARAMÈTRES ---
steps = 200
taux_erreur = 0.025
nb_frames = 100 # Nombre total d'images
duree_voulue = 10 # secondes

# Calcul du FPS pour que la vidéo dure exactement 10s (nb_frames / duree)
fps_calcule = nb_frames / duree_voulue

def animate(frame):
    gain = frame * 0.003
    rho_std = np.array([[1, 0], [0, 0]], dtype=complex)
    rho_lq = rho_std.copy()
    f_std, f_lq = [], []

    for i in range(steps):
        bruit = (1 - taux_erreur) * rho_std + taux_erreur * (I / 2.0)
        rho_std = bruit / np.trace(bruit)
        rho_lq = (1 - taux_erreur) * rho_lq + taux_erreur * (I / 2.0)

        x3 = np.real(np.trace(rho_lq @ Z))
        x0 = np.real(np.trace(rho_lq @ I))
        s2 = x0**2 - x3**2

        if s2 > 0.0001:
            eta = np.arctanh(min(0.999, 1 - s2))
            boost = expm(eta * gain * Z)
            rho_lq = boost @ rho_lq @ boost.conj().T
            rho_lq /= np.trace(rho_lq)

        f_std.append(calcul_purete(rho_std))
        f_lq.append(calcul_purete(rho_lq))

    line_std.set_data(range(steps), f_std)
    line_lq.set_data(range(steps), f_lq)
    text_gain.set_text(f"PUISSANCE DU BOOST : {gain:.3f}\nEFFICACITÉ : {((f_lq[-1]-0.5)/0.5)*100:.1f}%")
    return line_std, line_lq, text_gain

# interval=100 signifie 10 images par seconde durant la prévisualisation
ani = FuncAnimation(fig, animate, frames=nb_frames, interval=100, blit=True)

# --- SAUVEGARDE ---
filename = 'Demo_LorentzQ_Lente.mp4'
print(f"Génération d'une vidéo de {duree_voulue}s ({fps_calcule} FPS)...")

try:
    # On utilise le FPS calculé pour garantir la durée de 10s
    ani.save(filename, writer='ffmpeg', fps=fps_calcule)
    print("Vidéo prête. Téléchargement...")
    files.download(filename)
except Exception as e:
    print(f"Erreur FFMPEG, passage au format GIF...")
    filename_gif = filename.replace('.mp4', '.gif')
    ani.save(filename_gif, writer='pillow', fps=fps_calcule)
    files.download(filename_gif)

plt.close()

Génération d'une vidéo de 10s (10.0 FPS)...
Vidéo prête. Téléchargement...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [9]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from scipy.linalg import expm
import os
from google.colab import files

# --- INITIALISATION PHYSIQUE ---
I = np.eye(2, dtype=complex)
X = np.array([[0, 1], [1, 0]], dtype=complex)
Z = np.array([[1, 0], [0, -1]], dtype=complex)

def calcul_purete(rho):
    return np.real(np.trace(rho @ rho))

# Configuration Figure
plt.style.use('dark_background')
fig, ax = plt.subplots(figsize=(12, 7))
ax.set_xlim(0, 200)
ax.set_ylim(0.45, 1.05)

# Suppression des noms de marque -> Titres descriptifs et neutres
line_std, = ax.plot([], [], color='#ff4b4b', linestyle='--', label='Évolution sous Bruit (Markov)', alpha=0.8)
line_lq, = ax.plot([], [], color='#00d1ff', linewidth=3, label='Correction par Boost de Lorentz')
ax.axhline(y=1.0, color='#00ff41', linestyle=':', label='État Pur Théorique')

text_gain = ax.text(105, 0.55, '', fontsize=12, color='white', bbox={'facecolor': '#333333', 'alpha': 0.8})

# Titre académique
ax.set_title("Simulation de la Stabilisation de l'Information Quantique via Transformation de Lorentz", fontsize=14, pad=20)
ax.set_xlabel("Temps (Cycles de calcul)", fontsize=12)
ax.set_ylabel("Pureté de l'État Quantique (Tr[ρ²])", fontsize=12)
ax.legend(loc='lower left', fontsize=10)
ax.grid(color='gray', linestyle='--', alpha=0.2)

# --- PARAMÈTRES ---
steps = 200
taux_erreur = 0.025
nb_frames = 100
duree_voulue = 10 # 10 secondes pile

fps_calcule = nb_frames / duree_voulue

def animate(frame):
    gain = frame * 0.003
    rho_std = np.array([[1, 0], [0, 0]], dtype=complex)
    rho_lq = rho_std.copy()
    f_std, f_lq = [], []

    for i in range(steps):
        bruit = (1 - taux_erreur) * rho_std + taux_erreur * (I / 2.0)
        rho_std = bruit / np.trace(bruit)
        rho_lq = (1 - taux_erreur) * rho_lq + taux_erreur * (I / 2.0)

        x3 = np.real(np.trace(rho_lq @ Z))
        x0 = np.real(np.trace(rho_lq @ I))
        s2 = x0**2 - x3**2

        if s2 > 0.0001:
            eta = np.arctanh(min(0.999, 1 - s2))
            boost = expm(eta * gain * Z)
            rho_lq = boost @ rho_lq @ boost.conj().T
            rho_lq /= np.trace(rho_lq)

        f_std.append(calcul_purete(rho_std))
        f_lq.append(calcul_purete(rho_lq))

    line_std.set_data(range(steps), f_std)
    line_lq.set_data(range(steps), f_lq)
    # Texte purement descriptif
    text_gain.set_text(f"Paramètre de Boost η' : {gain:.3f}\nRestauration : {((f_lq[-1]-0.5)/0.5)*100:.1f}%")
    return line_std, line_lq, text_gain

ani = FuncAnimation(fig, animate, frames=nb_frames, interval=100, blit=True)

# --- SAUVEGARDE ---
filename = 'Simulation_Physique_Lorentz.mp4'
print(f"Génération de la vidéo ({duree_voulue}s)...")

try:
    ani.save(filename, writer='ffmpeg', fps=fps_calcule)
    files.download(filename)
except Exception as e:
    filename_gif = filename.replace('.mp4', '.gif')
    ani.save(filename_gif, writer='pillow', fps=fps_calcule)
    files.download(filename_gif)

plt.close()

Génération de la vidéo (10s)...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>